In [ ]:
import numpy as np
import pandas as pd
from fastai import *
from fastai.text import *

from clinical_note_utils import *

In [ ]:
valid_roc = []
test_roc = []

In [ ]:
path = "/home/littlefield/MIMIC-NLP/readmission-prediction/data/"
bs = 64

In [ ]:
clinical_data = load_data(path, 'clas_export-class.pkl', bs=bs)

In [ ]:
learn = text_classifier_learner(clinical_data, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, AUROC(), Precision(), Recall()]).to_fp16()
learn.load_encoder('clinical_lm-class-step2_encoder')

In [ ]:
learn.load("c_learner-1-4")

In [ ]:
preds = learn.get_preds(ds_type=DatasetType.Valid)

In [ ]:
def eval_model(inp, preds, ds="Validation", thresh=None):
    best_thresh = thresh
    if thresh is None:
        print("Theshold is not provided, calculating...")
        best_thresh = J_statistic(inp, preds)
    final_preds = np.array([1 if p > best_thresh else 0 for p in preds])
    acc = pos_accuracy(learn.data.valid_ds.y.items, final_preds, thresh=best_thresh)
    auc_score = auc(inp, preds)
    f1, precision, recall = f1_precision_recall(inp, final_preds)
    
    print("================================", ds, "Metrics for Postive Class ================================")
    print("Best Threshold:", best_thresh)
    print("Positive Class Acc.:", acc)
    print("AUC:", auc_score)
    print("F1 Score:", f1)
    print("Precision:", precision)
    print("Recall:", recall)
    
    scores = {"best_thresh": best_thresh,
              "acc": acc,
              "auc": auc_score,
              "f1": f1, 
              "precision": precision,
              "recall": recall}
    
    return scores


In [ ]:
valid_metrics_1 = eval_model(learn.data.valid_ds.y.items, preds[0][:, 1])
valid_roc.append(valid_metrics_1["auc"])

In [ ]:
test = (TextList.from_folder(path).filter_by_folder("test"))
learn.data.add_test(test)
t_preds = learn.get_preds(ds_type=DatasetType.Test)

# Create test labels based on order fast.ai read in test files
test_labels = []

for n in test.items:
    if "pos" in str(n):
        test_labels.append(1)
    else:
        test_labels.append(0)

In [ ]:
test_metrics_1 = eval_model(test_labels, t_preds[0][:, 1], ds="Test", thresh=valid_metrics_1["best_thresh"])
test_roc.append(test_metrics_1["auc"])

In [ ]:
learn.load("c_learner-2-4")

In [ ]:
preds = learn.get_preds(ds_type=DatasetType.Valid)
valid_metrics_2 = eval_model(learn.data.valid_ds.y.items, preds[0][:, 1])
valid_roc.append(valid_metrics_2["auc"])

test_metrics_2 = eval_model(test_labels, t_preds[0][:, 1], ds="Test", thresh=valid_metrics_2["best_thresh"])
test_roc.append(test_metrics_2["auc"])

In [ ]:
learn.load("c_learner-3-4")

preds = learn.get_preds(ds_type=DatasetType.Valid)
valid_metrics_3 = eval_model(learn.data.valid_ds.y.items, preds[0][:, 1])
valid_roc.append(valid_metrics_3["auc"])

test_metrics_3 = eval_model(test_labels, t_preds[0][:, 1], ds="Test", thresh=valid_metrics_3["best_thresh"])
test_roc.append(test_metrics_3["auc"])

In [ ]:
learn.load("c_learner-unfreeze-4")

preds = learn.get_preds(ds_type=DatasetType.Valid)
valid_metrics_4 = eval_model(learn.data.valid_ds.y.items, preds[0][:, 1])
valid_roc.append(valid_metrics_4["auc"])

test_metrics_4 = eval_model(test_labels, t_preds[0][:, 1], ds="Test", thresh=valid_metrics_4["best_thresh"])
test_roc.append(test_metrics_4["auc"])

In [ ]:
best_auc = np.argmax(valid_roc)
print(valid_roc[best_auc])
print(test_roc[best_auc])

In [ ]:
best_auc

In [ ]:
from sklearn.metrics import confusion_matrix
final_preds = [1 if p > valid_metrics_4["best_thresh"] else 0 for p in t_preds[0][:, 1]]

In [ ]:
confusion_matrix(test_labels, final_preds)

In [ ]:
4078/(4014+2326)

In [ ]:
297/450